In [1]:
using Pkg
Pkg.activate("libs/")
using JLD2
using DICOM
using Dates
using XLSX
using Plots
using ImageView
using Images
using Statistics

BAC_root_dir = "/home/molloi-lab/Desktop/Project BAC/BAC_Full_Data"
excel_path = "/home/molloi-lab/Desktop/Project BAC/BAC project/Ca mass calculation Final.xlsx"
V_P_dict = Dict("LCC" => 1, "LMLO" => 2, "RCC" => 3, "RMLO" => 4)
strs = ["LCC", "LMLO", "RCC", "RMLO"]

image_dirs = ["Patientstudy_BAC_2013a/data_from_KP", 
"Patientstudy_BAC_2013b/data_from_KP", 
"Patientstudy_BAC_2014/data_from_KP", 
"Patientstudy_BAC_2015a/data_from_KP", 
"Patientstudy_BAC_2015b/data_from_KP", 
"Patientstudy_BAC_2016/Data from KP"]

save_dir = "/media/molloi-lab/System Files/BAC_dataset/Clean_Dataset_full"
save_csv_dir = "/media/molloi-lab/System Files/BAC_dataset/splited_data.csv"
train_output_dir = "/media/molloi-lab/System Files/BAC_dataset/dataset_for_ML_full"
patch_size = 256
patch_size_half = round(Int, patch_size/2)

SIDs = readdir(save_dir)
num_SIDs = size(SIDs)[1];

  Activating project at `~/Desktop/Project BAC/BAC project/libs`


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/molloi-lab/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/molloi-lab/snap/code/common/.cache/gio-modules/libgiolibproxy.so


# Helper functions

In [2]:
function clean_directory(directory::String)
    for item in readdir(directory)
        item_path = joinpath(directory, item)
        rm(item_path, recursive=true)
    end
end

function zoom_pixel_values(img)
    a, b = minimum(img), maximum(img)
    img_ = (img .- a) ./ (b - a)
    return img_
end

function normalize_img(img)
    m = maximum(img)
    img = m .- img
    a = mean(img)
    s = std(img)
    img = (img .- a) ./ s 
    # println("mean = $(mean(img)), std = $(std(img))")
    return img
end

normalize_img (generic function with 1 method)

In [13]:
function save_image(curr_save_path, file_name, image_patch, label_patch)
    # save image
    @save joinpath(curr_save_path, "img", file_name*"jld") image_patch
    # save lable
    @save joinpath(curr_save_path, "lbl", file_name*"jld") label_patch
end

function super_sample_and_save_patch(img, lbl, prob_non_BAC, prob_BAC, super_rate, f_n, currkey_str, x_start, x_end, y_start, y_end)
    ct = 0
    # see if this has BAC
    is_BAC = sum(lbl[x_start:x_end, y_start:y_end]) > 0 ? true : false
    BAC_str = is_BAC ? "BAC" : "non-BAC"
    prob = is_BAC ? prob_BAC : prob_non_BAC

    # get patch
    curr_image = img[x_start:x_end, y_start:y_end]
    curr_label = lbl[x_start:x_end, y_start:y_end]

    # save patch
    file_name_ = f_n * ".$x_start.$x_end.$y_start.$y_end."
    curr_save_path = joinpath(train_output_dir, currkey_str, BAC_str)
    next_idx = 1

    # super sample
    if is_BAC
        for super = 1 : super_rate
            ct += 1
            file_name = file_name_ * "$super."
            save_image(curr_save_path, file_name, curr_image, curr_label)
        end
        next_idx = super_rate + 1
    end
    # random pick
    is_choosen = rand() < prob
    if is_choosen
        ct += 1
        file_name = file_name_ * "$next_idx."
        save_image(curr_save_path, file_name, curr_image, curr_label)
    end
    # return ct
    if is_BAC
        return 0, ct
    end
    return ct, 0
end

function patch_image(img, mask, lbl, prob_non_BAC, prob_BAC, super_rate, f_n, currkey_str)
    img = normalize_img(img)
    s = size(img)
    x = ceil(Int, s[1]/patch_size) + floor(Int, (s[1]-patch_size_half)/patch_size)
    y = ceil(Int, s[2]/patch_size) + floor(Int, (s[2]-patch_size_half)/patch_size)
    
    ct_empty , ct_non_empty = 0, 0
    for i = 1 : x-1
        x_start = 1+(i-1)*patch_size_half
        x_end = x_start+patch_size-1
        for j = 1 : y-1
            y_start = 1+(j-1)*patch_size_half
            y_end = y_start+patch_size-1
            # check patch
            if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
                ct1, ct2 = super_sample_and_save_patch(img, lbl, prob_non_BAC, prob_BAC, super_rate, f_n, currkey_str, x_start, x_end, y_start, y_end)
                ct_empty += ct1
                ct_non_empty += ct2
            end
        end
        # right col
        y_start, y_end = s[2]-patch_size+1, s[2]
        # check patch
        if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
            ct1, ct2 = super_sample_and_save_patch(img, lbl, prob_non_BAC, prob_BAC, super_rate, f_n, currkey_str, x_start, x_end, y_start, y_end)
            ct_empty += ct1
            ct_non_empty += ct2
        end
    end
    # last row
    x_start, x_end = s[1]-patch_size+1, s[1]
    for j = 1 : y-1
        y_start = 1+(j-1)*patch_size_half
        y_end = y_start+patch_size-1
        # check patch
        if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
            ct1, ct2 = super_sample_and_save_patch(img, lbl, prob_non_BAC, prob_BAC, super_rate, f_n, currkey_str, x_start, x_end, y_start, y_end)
            ct_empty += ct1
            ct_non_empty += ct2
        end
    end
    # right col
    y_start, y_end = s[2]-patch_size+1, s[2]
    # check patch
    if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
        ct1, ct2 = super_sample_and_save_patch(img, lbl, prob_non_BAC, prob_BAC, super_rate, f_n, currkey_str, x_start, x_end, y_start, y_end)
        ct_empty += ct1
        ct_non_empty += ct2
    end
    return ct_empty, ct_non_empty
end

patch_image (generic function with 1 method)

# 1. Build dataset

We need to know how many images and patches we have.

In [4]:
# # get total number of images
# total_number_images = zeros(Int, (4, 3))

# for i = 1 : num_SIDs
#     SID = SIDs[i]
#     files = readdir(joinpath(save_dir, SID))
#     for f in files
#         splited = split(f, '.') 
#         if splited[end] == "dcm"
#             # total ct
#             splited_ = split(splited[1], '_') 
#             currkey = V_P_dict[splited_[1] * splited_[2]]
#             total_number_images[currkey, 1] += 1
#             # patch ct
#             ct_empty, ct_non_empty = split(splited[end-1], '_')
#             total_number_images[currkey, 2] += parse(Int, ct_empty)
#             total_number_images[currkey, 3] += parse(Int, ct_non_empty)
#         end
#     end
# end
# println("       img_ct  non_BAC    BAC")
# for i = 1:4
#     print(strs[i],":\t")
#     for j = 1:3
#         print(total_number_images[i, j],"   ")
#     end
#     println()
# end

Total number of BAC patches = 103085  
Total number of wihout BAC patches = 5122436

|   Train+Valid   | img count | non-BAC patches   | BAC patches |
|------|--------|----------|------|
| LCC  |  5111  | 1200903  | 23272|
| LMLO |  5116  | 1376274  | 26455|
| RCC  |  5112  | 1173377  | 24514|
| RMLO |  5113  | 1371882  | 28844|

Now, Pick some patients for the validation set:

In [5]:
# target_num_BAC_patch_each_VP_valid_Set = 2500
# choose_poss = target_num_BAC_patch_each_VP_valid_Set / mean(total_number_images[:, 3])

In [6]:
# valid_pool, train_pool = [] , [] # SIDs' indexes
# total_number_images_valid = zeros(Int, (4, 3))
# total_number_images_train = zeros(Int, (4, 3))

# for i = 1 : num_SIDs
#     SID = SIDs[i]
#     is_choosen = rand() < choose_poss
#     if is_choosen
#         push!(valid_pool, i)
#     else
#         push!(train_pool, i)
#     end
#     files = readdir(joinpath(save_dir, SID))
#     for f in files
#         splited = split(f, '.') 
#         if splited[end] == "dcm"
#             splited_ = split(splited[1], '_') 
#             currkey = V_P_dict[splited_[1] * splited_[2]]
#             ct_empty, ct_non_empty = split(splited[end-1], '_')

#             if is_choosen
#                 # total ct
#                 total_number_images_valid[currkey, 1] += 1
#                 # patch ct
#                 total_number_images_valid[currkey, 2] += parse(Int, ct_empty)
#                 total_number_images_valid[currkey, 3] += parse(Int, ct_non_empty)
#             else
#                 # total ct
#                 total_number_images_train[currkey, 1] += 1
#                 # patch ct
#                 total_number_images_train[currkey, 2] += parse(Int, ct_empty)
#                 total_number_images_train[currkey, 3] += parse(Int, ct_non_empty)
#             end
#         end
#     end
# end
# println(size(valid_pool)[1], " patients in valid set.")
# println("       img_ct  non_BAC    BAC")
# for i = 1:4
#     print(strs[i],":\t")
#     for j = 1:3
#         print(total_number_images_valid[i, j],"   ")
#     end
#     println()
# end
# println("\n", size(train_pool)[1], " patients in train set.")
# println("       img_ct  non_BAC    BAC")
# for i = 1:4
#     print(strs[i],":\t")
#     for j = 1:3
#         print(total_number_images_train[i, j],"   ")
#     end
#     println()
# end

Save SIDs info to csv:

In [7]:
# valid_SIDs, train_SIDs = SIDs[valid_pool], SIDs[train_pool]
# @save "data_loader_pools_SIDs_2.jld" valid_pool train_pool valid_SIDs train_SIDs total_number_images_train

In [8]:
@load "data_loader_pools_SIDs.jld" valid_pool train_pool valid_SIDs train_SIDs total_number_images_train

5-element Vector{Symbol}:
 :valid_pool
 :train_pool
 :valid_SIDs
 :train_SIDs
 :total_number_images_train

Now, blance the train set.

In [9]:
# generate supersampling and downsampling rate
target_num_BAC_patch_each_VP_train_Set = 50000
oversample_factor = floor.(Int, 50000 ./ total_number_images_train[:, 3])
choose_poss_BAC, choose_poss_non_BAC = zeros(4), zeros(4)
for i = 1:4
    # BAC
    choose_poss_BAC[i] = (target_num_BAC_patch_each_VP_train_Set / total_number_images_train[i, 3]) - oversample_factor[i]
    # non-BAC
    curr_ct = total_number_images_train[i, 2]
    choose_poss_non_BAC[i] = target_num_BAC_patch_each_VP_train_Set / curr_ct
end

# 2. Save patches to local.

In [15]:
# runtime ~= 15min
# Create and clean folder
for i = 1:4
    curr_dir = joinpath(train_output_dir, strs[i])
    isdir(curr_dir) || mkdir(curr_dir)
    clean_directory(curr_dir)

    curr_dir_ = joinpath(curr_dir, "BAC")
    isdir(curr_dir_) || mkdir(curr_dir_)
    clean_directory(curr_dir_)

    curr_dir__ = joinpath(curr_dir_, "img")
    isdir(curr_dir__) || mkdir(curr_dir__)
    clean_directory(curr_dir__)
    curr_dir__ = joinpath(curr_dir_, "lbl")
    isdir(curr_dir__) || mkdir(curr_dir__)
    clean_directory(curr_dir__)

    curr_dir_ = joinpath(curr_dir, "non-BAC")
    isdir(curr_dir_) || mkdir(curr_dir_)
    clean_directory(curr_dir_)

    curr_dir__ = joinpath(curr_dir_, "img")
    isdir(curr_dir__) || mkdir(curr_dir__)
    clean_directory(curr_dir__)
    curr_dir__ = joinpath(curr_dir_, "lbl")
    isdir(curr_dir__) || mkdir(curr_dir__)
    clean_directory(curr_dir__)
end
@info "Started"
# Let's go
tmp_s = size(train_SIDs)[1]
total_number_images_train_ = zeros(Int, (tmp_s, 8)) # "LCC-non", "LCC-BAC", "LMLO-non", "LMLO-BAC", "RCC-non", "RCC-BAC", "RMLO-non", "RMLO-BAC"
Threads.@threads for i = 1 : tmp_s
# for i = 1 : 1
    SID = train_SIDs[i]
    files = readdir(joinpath(save_dir, SID))
    for f in files
        splited = split(f, '.') 
        if splited[end] == "dcm"
            # set up path
            img_path = joinpath(save_dir, SID, f)
            lbl_path = joinpath(save_dir, SID, splitext(f)[1]*".png")
            # get V_P idx
            splited_ = split(splited[1], '_') 
            currkey_str = splited_[1] * splited_[2]
            currkey_idx = V_P_dict[currkey_str]
            f_n = SID *"."*splited[1]
            # read dicom image
            img = Float32.(dcm_parse(img_path)[(0x7fe0, 0x0010)])
            # read label
            lbl = Float32.(Images.load(lbl_path))
            # get mask
            mask = Float32.(1 .- round.(zoom_pxiel_values(img)))
            # get probabilities
            prob_non_BAC = choose_poss_non_BAC[currkey_idx]
            prob_BAC = choose_poss_BAC[currkey_idx]
            super_rate = oversample_factor[currkey_idx]
            # get num of batch
            ct_non_BAC, ct_BAC = patch_image(img, mask, lbl, prob_non_BAC, prob_BAC, super_rate, f_n, currkey_str)
            total_number_images_train_[i, 2*(currkey_idx-1)+1 ] += ct_non_BAC
            total_number_images_train_[i, 2*(currkey_idx-1)+2 ] += ct_BAC
        end
    end
    GC.gc(true)
end

# visualize data
total_number_images_train = zeros(Int, (4, 2))
for i = 1:tmp_s
    rslt = total_number_images_train_[i, :, :]
    for j = 1 : 4
        for k = 1 : 2
            total_number_images_train[j, k] += rslt[(j-1)*2+k]
        end
    end
end

println("       non_BAC    BAC")
for i = 1:4
    print(strs[i],":\t  ")
    for j = 1:2
        print(total_number_images_train[i, j],"        ")
    end
    println()
end

┌ Info: Started
└ @ Main /home/molloi-lab/Desktop/Project BAC/BAC project/1.4_split_data.ipynb:30


       non_BAC    BAC
LCC:	  49772        49910        
LMLO:	  50154        49980        
RCC:	  49963        50039        
RMLO:	  49561        49952        


|   Train   | non-BAC patches   | BAC patches |
|------|----------|------|
| LCC  | 49772  | 49910|
| LMLO | 50154  | 49980|
| RCC  | 49963  | 50039|
| RMLO | 49561  | 49952|